In [1]:
# 0:vcd, 2:glc, 3:lac, 4:amm, 7:igg
tn = 7
rt = 2

# **Base**

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/AI Bioprocess/Data/~220915
!pwd
!ls
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
/content/drive/MyDrive/AI Bioprocess/Data/~220915
/content/drive/MyDrive/AI Bioprocess/Data/~220915
ammonia.csv	igg_qc.csv	     lactate.csv	 ph_rt.csv
ammonia_qc.csv	igg_rt.csv	     lactate_qc.csv	 vcd.csv
ammonia_rt.csv	input_ammonia.csv    lactate_rt.csv	 vcd_qc.csv
do.csv		input_do.csv	     output_ammonia.csv  vcd_rt.csv
do_qc.csv	input_glucose.csv    output_glucose.csv  viability.csv
do_rt.csv	input_igg.csv	     output_igg.csv	 viability_qc.csv
glucose.csv	input_lactate.csv    output_lactate.csv  viability_rt.csv
glucose_qc.csv	input_ph.csv	     output_vcd.csv
glucose_rt.csv	input_vcd.csv	     ph.csv
igg.csv		input_viability.csv  ph_qc.csv


In [3]:
hp_svm     = [[5,2],[],[1,0.1],[1,0.1],[1,0.1],[],[],[1,0.1]]
hp_ann_lr  = [0.1, 0, 0.01,  0.03, 0.03, 0,0, 0.03]
hp_rnn_lr  = [0.1, 0, 0.008, 0.03, 0.03, 0,0, 0.03]
hp_lstm_lr = [0.5, 0, 0.5,   0.03, 0.03, 0,0, 0.03]
hp_gru_lr  = [2.5, 0, 0.1,   0.03, 0.03, 0,0, 0.03]

hp_lr = [0,0,0,0,0,0,0,0,0,0,0,0,hp_ann_lr[tn],hp_rnn_lr[tn],hp_lstm_lr[tn],hp_gru_lr[tn]]

units = ['($10^6$ cells/mL)',0,'(g/L)','(g/L)','(mM)',0,0,'(g/L)']

In [4]:
import os
import random
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.cross_decomposition import PLSRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.multioutput import MultiOutputRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as rse


import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import combinations
import time
import warnings
from math import ceil

In [5]:
# Functions

def seed_fix():
  seed = 1337
  os.environ['PYTHONHASHSEED'] = str(seed)
  os.environ['TF_DETERMINISTIC_OPS'] = '1'
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  tf.compat.v1.set_random_seed(seed)
  config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=config)
  K.set_session(sess)

# 모델 준비
seed_fix()
def build_LR():
  model = LinearRegression()
  return model

def build_LASSO():
  model = Lasso(alpha=0.03)
  return model

def build_EN():
  model = ElasticNet(alpha=0.03)
  return model

def build_KNN():
  model = KNeighborsRegressor()
  return model

def build_CART():
  model = DecisionTreeRegressor()
  return model

def build_GBM():
  model = MultiOutputRegressor(GradientBoostingRegressor())
  return model

def build_XG():
  model = MultiOutputRegressor(XGBRegressor())
  return model

def build_ADA():
  model = MultiOutputRegressor(AdaBoostRegressor())
  return model

def build_SVM():
  model = MultiOutputRegressor(SVR(C=hp_svm[tn][0],epsilon = hp_svm[tn][1]))
  return model

def build_RF():
  model = RandomForestRegressor()
  return model

def build_PLS():
  model = PLSRegression()
  return model

def build_GPR():
  kernel = DotProduct() + WhiteKernel()
  model = GaussianProcessRegressor(kernel=kernel)
  return model

def build_ANN(input_shape,layer_1,layer_2,output_num,Learning_rate):
  seed=1337
  model = keras.Sequential([
                            layers.Dense(layer_1, activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=seed),input_shape=(input_shape,)),
                            layers.Dense(layer_2, activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=seed)),
                            layers.Dense(output_num)
                            ])
  
  optimizer = tf.keras.optimizers.RMSprop(Learning_rate)
  model.compile(loss='mae',
                optimizer=optimizer,
                metrics=['mae','mse']
                )
  return model

def build_RNN(input_feature,units,output_feature,time_step,Learning_rate):
  seed=1337
  model = keras.Sequential()
  model.add(keras.layers.SimpleRNN(units,kernel_initializer=keras.initializers.HeNormal(seed=seed),input_shape=(time_step,input_feature),stateful=False))
  model.add(keras.layers.Dropout(rate=0.1))
  model.add(keras.layers.Dense(units=output_feature,kernel_initializer=keras.initializers.HeNormal(seed=seed)))
  optimizer = tf.keras.optimizers.Adam(learning_rate=Learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
  model.compile(loss='mae', optimizer=optimizer)
  return model

def build_LSTM(input_feature,units,output_feature,time_step,Learning_rate):
  seed=1337
  model = keras.Sequential()
  model.add(keras.layers.LSTM(units,kernel_initializer=keras.initializers.HeNormal(seed=seed),input_shape=(time_step,input_feature),stateful=False))
  model.add(keras.layers.Dropout(rate=0.1))
  model.add(keras.layers.Dense(units=output_feature,kernel_initializer=keras.initializers.HeNormal(seed=seed)))
  optimizer = tf.keras.optimizers.Adam(learning_rate=Learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
  model.compile(loss='mae', optimizer=optimizer)
  return model

def build_GRU(input_feature,units,output_feature,time_step,Learning_rate):
  seed=1337
  model = keras.Sequential()
  model.add(keras.layers.GRU(units,kernel_initializer=keras.initializers.HeNormal(seed=seed),input_shape=(time_step,input_feature),stateful=False))
  model.add(keras.layers.Dropout(rate=0.1))
  model.add(keras.layers.Dense(units=output_feature,kernel_initializer=keras.initializers.HeNormal(seed=seed)))
  optimizer = tf.keras.optimizers.Adam(learning_rate=Learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False)
  model.compile(loss='mae', optimizer=optimizer)
  return model

def make_recursive(data):
  X = np.zeros((data.shape[0]*(data.shape[1]-3),3))
  y = np.zeros((data.shape[0]*(data.shape[1]-3),1))
  for i in range(data.shape[0]):
    for j in range(data.shape[1]-3):
      X[(5*i+j):(5*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
      y[(5*i+j):(5*i+j+1),0:1] = data[i:(i+1),(j+3):(j+4)]
  return X.astype(float), y.astype(float)

def make_direct(data,future):
  X = np.zeros((data.shape[0]*(data.shape[1]-2-future),3))
  y = np.zeros((data.shape[0]*(data.shape[1]-2-future),1))
  for i in range(data.shape[0]):
    for j in range(data.shape[1]-2-future):
      X[((data.shape[1]-2-future)*i+j):((data.shape[1]-2-future)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
      y[((data.shape[1]-2-future)*i+j):((data.shape[1]-2-future)*i+j+1),0:1] = data[i:(i+1),(j+2+future):(j+3+future)]
  return X.astype(float), y.astype(float)

def make_mimo(data,a):
  if a == 2:
    X = np.zeros((data.shape[0],4))
    y = np.zeros((data.shape[0],4))
    for i in range(data.shape[0]):
      X[i:(i+1),0:4] = data[i:(i+1),0:4]
      y[i:(i+1),0:4] = data[i:(i+1),4:8]
  elif a == 3:
    X = np.zeros((data.shape[0]*(a-1),4))
    y = np.zeros((data.shape[0]*(a-1),5-a+1))
    for i in range(data.shape[0]):
      for j in range(a-1):
        X[((a-1)*i+j):((a-1)*i+j+1),0:4] = data[i:(i+1),j:j+4]
        y[(a-1)*i+j:((a-1)*i+j+1),0:5-a+1] = data[i:(i+1),j+4:j+9-a+1]
  elif a == 4:
    X = np.zeros((data.shape[0]*(a-1),4))
    y = np.zeros((data.shape[0]*(a-1),5-a+1))
    for i in range(data.shape[0]):
      for j in range(a-1):
        X[((a-1)*i+j):((a-1)*i+j+1),0:4] = data[i:(i+1),j:j+4]
        y[(a-1)*i+j:((a-1)*i+j+1),0:5-a+1] = data[i:(i+1),j+4:j+9-a+1]

  return X.astype(float), y.astype(float)

def make_dirrec(data,future):
  X = np.zeros((data.shape[0]*(data.shape[1]-2-future),2+future))
  y = np.zeros((data.shape[0]*(data.shape[1]-2-future),1))
  for i in range(data.shape[0]):
    for j in range(data.shape[1]-2-future):
      X[((data.shape[1]-2-future)*i+j):((data.shape[1]-2-future)*i+j+1),0:(2+future)] = data[i:(i+1),j:(j+future+2)]
      y[((data.shape[1]-2-future)*i+j):((data.shape[1]-2-future)*i+j+1),0:1] = data[i:(i+1),(j+2+future):(j+3+future)]
  return X.astype(float), y.astype(float)

def make_dirmo3(data,a):
  if a == 1:
    X = np.zeros((data.shape[0]*(data.shape[1]-5),3))
    y = np.zeros((data.shape[0]*(data.shape[1]-5),3))
    for i in range(data.shape[0]):
      for j in range(data.shape[1]-5):
        X[((data.shape[1]-5)*i+j):((data.shape[1]-5)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
        y[((data.shape[1]-5)*i+j):((data.shape[1]-5)*i+j+1),0:3] = data[i:(i+1),(j+3):(j+6)]
  elif a == 2 or a == 3:
    X = np.zeros((data.shape[0]*(data.shape[1]-4),3))
    y = np.zeros((data.shape[0]*(data.shape[1]-4),2))
    for i in range(data.shape[0]):
      for j in range(data.shape[1]-4):
        X[((data.shape[1]-4)*i+j):((data.shape[1]-4)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
        y[((data.shape[1]-4)*i+j):((data.shape[1]-4)*i+j+1),0:2] = data[i:(i+1),(j+3):(j+5)]
  elif a == 4:
    X = np.zeros((data.shape[0]*(data.shape[1]-3),3))
    y = np.zeros((data.shape[0]*(data.shape[1]-3),1))
    for i in range(data.shape[0]):
      for j in range(data.shape[1]-3):
        X[((data.shape[1]-3)*i+j):((data.shape[1]-3)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
        y[((data.shape[1]-3)*i+j):((data.shape[1]-3)*i+j+1),0:1] = data[i:(i+1),(j+3):(j+4)]
  return X.astype(float), y.astype(float)

def make_dirmo2(data,a):
  if a == 1:
    X = np.zeros((data.shape[0]*(data.shape[1]-7),3))
    y = np.zeros((data.shape[0]*(data.shape[1]-7),2))
    for i in range(data.shape[0]):
      for j in range(data.shape[1]-7):
        X[((data.shape[1]-7)*i+j):((data.shape[1]-7)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
        y[((data.shape[1]-7)*i+j):((data.shape[1]-7)*i+j+1),0:2] = data[i:(i+1),(j+6):(j+8)]
  elif a == 2:
    X = np.zeros((data.shape[0]*(data.shape[1]-6),3))
    y = np.zeros((data.shape[0]*(data.shape[1]-6),2))
    for i in range(data.shape[0]):
      for j in range(data.shape[1]-6):
        X[((data.shape[1]-6)*i+j):((data.shape[1]-6)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
        y[((data.shape[1]-6)*i+j):((data.shape[1]-6)*i+j+1),0:2] = data[i:(i+1),(j+5):(j+7)]
  elif a == 3:
    X = np.zeros((data.shape[0]*(data.shape[1]-5),3))
    y = np.zeros((data.shape[0]*(data.shape[1]-5),1))
    for i in range(data.shape[0]):
      for j in range(data.shape[1]-5):
        X[((data.shape[1]-5)*i+j):((data.shape[1]-5)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
        y[((data.shape[1]-5)*i+j):((data.shape[1]-5)*i+j+1),0:1] = data[i:(i+1),(j+5):(j+6)]

  elif a ==4:
    X = np.zeros((data.shape[0]*(data.shape[1]-4),3))
    y = np.zeros((data.shape[0]*(data.shape[1]-4),1))
    for i in range(data.shape[0]):
      for j in range(data.shape[1]-4):
        X[((data.shape[1]-4)*i+j):((data.shape[1]-4)*i+j+1),0:3] = data[i:(i+1),j:(j+3)]
        y[((data.shape[1]-4)*i+j):((data.shape[1]-4)*i+j+1),0:1] = data[i:(i+1),(j+4):(j+5)]

  return X.astype(float), y.astype(float)


def make_ms_data(data,f,a,future):
  if f == 0:
    return make_recursive(data)
  elif f == 1:
    return make_direct(data,future)
  elif f == 2:
    return make_mimo(data,a)
  elif f == 3:
    return make_dirrec(data,future)
  elif f == 4:
    return make_dirmo3(data,a)


def algorithm_selection(a,input_num,output_num,input_feature,units,output_feature,time_step,Learning_rate):
  if a == 0:
    model = build_LR()
  elif a == 1:
    model = build_LASSO()
  elif a == 2:
    model = build_EN()
  elif a == 3:
    model = build_KNN()
  elif a == 4:
    model = build_CART()
  elif a == 5:
    model = build_GBM()
  elif a == 6:
    model = build_XG()
  elif a == 7:
    model = build_ADA()
  elif a == 8:
    model = build_SVM()
  elif a == 9:
    model = build_RF()
  elif a == 10:
    model = build_PLS()
  elif a == 11:
    model = build_GPR()
  elif a == 12:
    model = build_ANN(input_num,input_num,ceil(input_num/2),output_num,Learning_rate)
  elif a == 13:
    model = build_RNN(input_feature,units,output_feature,time_step,Learning_rate)
  elif a == 14:
    model = build_LSTM(input_feature,units,output_feature,time_step,Learning_rate)
  elif a == 15:
    model = build_GRU(input_feature,units,output_feature,time_step,Learning_rate)
  return model


def recursive_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # RT data
  data_train_input_rt = []
  for i in all_vcd_com:
    data_train_input_rt.append(rt_data[i].to_numpy()[:,1:3+rt])
  data_train_output_rt = []
  for i in out_list:
    data_train_output_rt.append(rt_data[i].to_numpy()[:,1:3+rt])

  #recursive
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_recursive(data_train_input[i])[0])
    y_train_stack.append(make_recursive(data_train_output[i])[1])
    X_test_stack.append(make_recursive(data_test_input[i])[0])
    y_test_stack.append(make_recursive(data_test_output[i])[1])
    X_train_stack[i] = np.concatenate([X_train_stack[i],make_recursive(data_train_input_rt[i][:,0:rt+2])[0]])
    y_train_stack[i] = np.concatenate([y_train_stack[i],make_recursive(data_train_output_rt[i][:,0:rt+2])[1]])
  if tn not in all_vcd_com:
    y_train_stack.append(make_recursive(data_train_output[len(all_vcd_com)])[1])
    y_test_stack.append(make_recursive(data_test_output[len(all_vcd_com)])[1])
    y_train_stack[-1] = np.concatenate([y_train_stack[-1],make_recursive(data_train_output_rt[-1][:,0:rt+2])[1]])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
 

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)
  
  if mn in [13,14,15]:
    for i in range(len(X_train_s_list)):
      X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
      X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],Learning_rate))

  if mn in range(12):
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn in [12,13,14,15]:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)
  
  address1 = []
  address2 = []
  address1 = list(np.arange(0,out_list.index(tn)))
  address2 = list(np.arange(out_list.index(tn)+len(out_list)-len(all_vcd_com), len(out_list)))
  address1.extend(address2)

  conditions = []
  ten_rd = np.concatenate(data_test_input)[:,0:2+rt].reshape(-1,4,2+rt)
  conditions.append(ten_rd[:,0,:])
  conditions.append(ten_rd[:,1,:])
  conditions.append(ten_rd[:,2,:])
  conditions.append(ten_rd[:,3,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:3+rt])
  y_pred_all = conditions.copy()

  for cn in range(4):
    VCD_tem[cn] = np.concatenate([VCD_tem[cn],np.zeros(shape=(VCD_tem[cn].shape[0],8-VCD_tem[cn].shape[1]))],1)
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  if mn in range(13):
    #recursive
    for cn in range(4):
      for day in range(rt-1,5):
        VCD_tem[cn][:,(day+3):(day+4)] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,day:(day+3)].reshape(1,VCD_tem[cn].shape[0]*3))).T[address1]
        y_pred_all[cn][day+3] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,day:(day+3)].reshape(1,VCD_tem[cn].shape[0]*3))).T[out_list.index(tn)]
  elif mn in [13,14,15]:
    #recursive RNN
    for cn in range(4):
      for day in range(rt-1,5):
        VCD_tem[cn][:,(day+3):(day+4)] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,day:(day+3)].reshape(1,VCD_tem[cn].shape[0]*3)).reshape(1,VCD_tem[cn][:,day:(day+3)].shape[0],VCD_tem[cn][:,day:(day+3)].shape[1]).transpose(0,2,1),verbose=0).T[address1]
        y_pred_all[cn][day+3] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,day:(day+3)].reshape(1,VCD_tem[cn].shape[0]*3)).reshape(1,VCD_tem[cn][:,day:(day+3)].shape[0],VCD_tem[cn][:,day:(day+3)].shape[1]).transpose(0,2,1),verbose=0).T[out_list.index(tn)]

  return y_pred_all, np.concatenate([y_pred_all[0][2+rt:8],y_pred_all[1][2+rt:8],y_pred_all[2][2+rt:8],y_pred_all[3][2+rt:8]]), models


def direct_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # RT data
  data_train_input_rt = []
  for i in all_vcd_com:
    data_train_input_rt.append(rt_data[i].to_numpy()[:,1:3+rt])
  data_train_output_rt = []
  for i in out_list:
    data_train_output_rt.append(rt_data[i].to_numpy()[:,1:3+rt])
  
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []
  # direct 1
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],1)[0])
    X_test_stack.append(make_direct(data_test_input[i],1)[0])
    if rt >= 2:
      X_train_stack[i] = np.concatenate([X_train_stack[i],make_direct(data_train_input_rt[i],1)[0]])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],1)[1])
  if rt >= 2:
    y_train_stack[0] = np.concatenate([y_train_stack[0],make_direct(data_train_output_rt[i],1)[1]])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],1)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 2
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],2)[0])
    X_test_stack.append(make_direct(data_test_input[i],2)[0])
    if rt >= 3:
      X_train_stack[i] = np.concatenate([X_train_stack[i],make_direct(data_train_input_rt[i],2)[0]])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],2)[1])
  if rt >= 3:
    y_train_stack[0] = np.concatenate([y_train_stack[0],make_direct(data_train_output_rt[i],2)[1]])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],2)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 3
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],3)[0])
    X_test_stack.append(make_direct(data_test_input[i],3)[0])
    if rt >= 4:
      X_train_stack[i] = np.concatenate([X_train_stack[i],make_direct(data_train_input_rt[i],3)[0]])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],3)[1])
  if rt >= 4:
    y_train_stack[0] = np.concatenate([y_train_stack[0],make_direct(data_train_output_rt[i],3)[1]])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],3)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 4
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],4)[0])
    X_test_stack.append(make_direct(data_test_input[i],4)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],4)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],4)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # direct 5
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_direct(data_train_input[i],5)[0])
    X_test_stack.append(make_direct(data_test_input[i],5)[0])
  y_train_stack.append(make_direct(data_train_output[out_list.index(tn)],5)[1])
  y_test_stack.append(make_direct(data_test_output[out_list.index(tn)],5)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)-rt+1):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  if mn in [13,14,15]:
    for i in range(len(X_train_s_list)):
      X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
      X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],Learning_rate))

  if mn in range(12):
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn in [12,13,14,15]:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  ten_rd = np.concatenate(data_test_input)[:,0:2+rt].reshape(-1,4,2+rt)
  conditions.append(ten_rd[:,0,:])
  conditions.append(ten_rd[:,1,:])
  conditions.append(ten_rd[:,2,:])
  conditions.append(ten_rd[:,3,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:3+rt])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  if mn in range(13):
    # direct
    for cn in range(4):
      for pm in range(rt-1,5):
        y_pred_all[cn][pm+3] = models[pm-rt+1].predict(scaler_list[pm-rt+1].transform(VCD_tem[cn][:,rt-1:rt+2].reshape(1,VCD_tem[cn].shape[0]*3))).reshape(1)
  
  elif mn in [13,14,15]:
    # direct RNN
    for cn in range(4):
      for pm in range(rt-1,5):
        y_pred_all[cn][pm+3] = models[pm-rt+1].predict(scaler_list[pm-rt+1].transform(VCD_tem[cn][:,rt-1:rt+2].reshape(1,VCD_tem[cn].shape[0]*3)).reshape(1,VCD_tem[cn][:,rt-1:rt+2].shape[0],VCD_tem[cn][:,rt-1:rt+2].shape[1]).transpose(0,2,1),verbose=0).reshape(1)

  return y_pred_all, np.concatenate([y_pred_all[0][rt+2:8],y_pred_all[1][rt+2:8],y_pred_all[2][rt+2:8],y_pred_all[3][rt+2:8]]), models


def mimo_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # RT data
  data_train_input_rt = []
  for i in all_vcd_com:
    data_train_input_rt.append(rt_data[i].to_numpy()[:,1:3+rt])
  data_train_output_rt = []
  for i in out_list:
    data_train_output_rt.append(rt_data[i].to_numpy()[:,1:3+rt])

  # MIMO
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_mimo(data_train_input[i],rt)[0])
    X_test_stack.append(make_mimo(data_test_input[i],rt)[0])
  y_train_stack.append(make_mimo(data_train_output[out_list.index(tn)],rt)[1])
  y_test_stack.append(make_mimo(data_test_output[out_list.index(tn)],rt)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  if mn in [13,14,15]:
    for i in range(len(X_train_s_list)):
      X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
      X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],Learning_rate))

  if mn in range(12):
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn in [12,13,14,15]:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  ten_rd = np.concatenate(data_test_input)[:,0:2+rt].reshape(-1,4,2+rt)
  conditions.append(ten_rd[:,0,:])
  conditions.append(ten_rd[:,1,:])
  conditions.append(ten_rd[:,2,:])
  conditions.append(ten_rd[:,3,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:3+rt])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  if mn in range(13):
    # MIMO
    for cn in range(4):
      y_pred_all[cn][2+rt:8] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,rt-2:rt+2].reshape(1,VCD_tem[cn].shape[0]*4))).reshape(6-rt)
  elif mn in [13,14,15]:
    # MIMO RNN
    for cn in range(4):
      y_pred_all[cn][2+rt:8] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,rt-2:rt+2].reshape(1,VCD_tem[cn].shape[0]*4)).reshape(1,VCD_tem[cn][:,0:4].shape[0],VCD_tem[cn][:,0:4].shape[1]).transpose(0,2,1),verbose=0).reshape(6-rt)

  return y_pred_all, np.concatenate([y_pred_all[0][2+rt:8],y_pred_all[1][2+rt:8],y_pred_all[2][2+rt:8],y_pred_all[3][2+rt:8]]), models


def dirrec_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # RT data
  data_train_input_rt = []
  for i in all_vcd_com:
    data_train_input_rt.append(rt_data[i].to_numpy()[:,1:3+rt])
  data_train_output_rt = []
  for i in out_list:
    data_train_output_rt.append(rt_data[i].to_numpy()[:,1:3+rt])


  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []
  # dirrec 1
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],1)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],1)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],1)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],1)[1])
    if rt >= 2:
      X_train_stack[i] = np.concatenate([X_train_stack[i],make_dirrec(data_train_input_rt[i],1)[0]])
      y_train_stack[i] = np.concatenate([y_train_stack[i],make_dirrec(data_train_output_rt[i],1)[1]])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],1)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],1)[1])
    if rt >= 2:
      y_train_stack[-1] = np.concatenate([y_train_stack[-1],make_dirrec(data_train_output_rt[-1],1)[1]])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 2
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],2)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],2)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],2)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],2)[1])
    if rt >= 3:
      X_train_stack[i] = np.concatenate([X_train_stack[i],make_dirrec(data_train_input_rt[i],2)[0]])
      y_train_stack[i] = np.concatenate([y_train_stack[i],make_dirrec(data_train_output_rt[i],2)[1]])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],2)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],2)[1])
    if rt >= 3:
      y_train_stack[-1] = np.concatenate([y_train_stack[-1],make_dirrec(data_train_output_rt[-1],2)[1]])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 3
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],3)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],3)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],3)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],3)[1])
    if rt >= 4:
      X_train_stack[i] = np.concatenate([X_train_stack[i],make_dirrec(data_train_input_rt[i],3)[0]])
      y_train_stack[i] = np.concatenate([y_train_stack[i],make_dirrec(data_train_output_rt[i],3)[1]])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],3)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],3)[1])
    if rt >= 4:
      y_train_stack[-1] = np.concatenate([y_train_stack[-1],make_dirrec(data_train_output_rt[-1],3)[1]])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 4
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],4)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],4)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],4)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],4)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],4)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],4)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))
  # dirrec 5
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirrec(data_train_input[i],5)[0])
    X_test_stack.append(make_dirrec(data_test_input[i],5)[0])
    y_train_stack.append(make_dirrec(data_train_output[i],5)[1])
    y_test_stack.append(make_dirrec(data_test_output[i],5)[1])
  if tn not in all_vcd_com:
    y_train_stack.append(make_dirrec(data_train_output[len(all_vcd_com)],5)[1])
    y_test_stack.append(make_dirrec(data_test_output[len(all_vcd_com)],5)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)-rt+1):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  if mn in [13,14,15]:
    for i in range(len(X_train_s_list)):
      X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
      X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],Learning_rate))

  if mn in range(12):
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn in [12,13,14,15]:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)  
  
  address1 = []
  address2 = []
  address1 = list(np.arange(0,out_list.index(tn)))
  address2 = list(np.arange(out_list.index(tn)+len(out_list)-len(all_vcd_com), len(out_list)))
  address1.extend(address2)

  conditions = []
  ten_rd = np.concatenate(data_test_input)[:,0:2+rt].reshape(-1,4,2+rt)
  conditions.append(ten_rd[:,0,:])
  conditions.append(ten_rd[:,1,:])
  conditions.append(ten_rd[:,2,:])
  conditions.append(ten_rd[:,3,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:3+rt])
  y_pred_all = conditions.copy()

  for cn in range(4):
    VCD_tem[cn] = np.concatenate([VCD_tem[cn],np.zeros(shape=(VCD_tem[cn].shape[0],8-VCD_tem[cn].shape[1]))],1)
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  if mn in range(13):
    for cn in range(4):
      for day in range(rt-1,5):
        VCD_tem[cn][:,(day+3):(day+4)] = models[day+1-rt].predict(scaler_list[day+1-rt].transform(VCD_tem[cn][:,(rt-1):(day+3)].reshape(1,-1))).T[address1]
        y_pred_all[cn][day+3] = models[day+1-rt].predict(scaler_list[day+1-rt].transform(VCD_tem[cn][:,(rt-1):(day+3)].reshape(1,-1))).T[out_list.index(tn)]

  elif mn in [13,14,15]:
    for cn in range(4):
      for day in range(rt-1,5):
        VCD_tem[cn][:,(day+3):(day+4)] = models[day+1-rt].predict(scaler_list[day+1-rt].transform(VCD_tem[cn][:,(rt-1):(day+3)].reshape(1,-1)).reshape(1,VCD_tem[cn][:,(rt-1):(day+3)].shape[0],VCD_tem[cn][:,(rt-1):(day+3)].shape[1]).transpose(0,2,1),verbose=0).T[address1]
        y_pred_all[cn][day+3] = models[day+1-rt].predict(scaler_list[day+1-rt].transform(VCD_tem[cn][:,(rt-1):(day+3)].reshape(1,-1)).reshape(1,VCD_tem[cn][:,(rt-1):(day+3)].shape[0],VCD_tem[cn][:,(rt-1):(day+3)].shape[1]).transpose(0,2,1),verbose=0).T[out_list.index(tn)]

  return y_pred_all, np.concatenate([y_pred_all[0][2+rt:8],y_pred_all[1][2+rt:8],y_pred_all[2][2+rt:8],y_pred_all[3][2+rt:8]]), models


def dirmo_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate):
  out_list = list(all_vcd_com).copy()
  if tn not in all_vcd_com:
    out_list.append(tn)
    out_list.sort()

  data_train_input = []
  data_test_input = []
  for i in all_vcd_com:
    data_train_input.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_input.append(raw_data[i][condition_test].to_numpy()[:,1:9])
  data_train_output = []
  data_test_output = []
  for i in out_list:
    data_train_output.append(raw_data[i][condition_train].to_numpy()[:,1:9])
    data_test_output.append(raw_data[i][condition_test].to_numpy()[:,1:9])

  # RT data
  data_train_input_rt = []
  for i in all_vcd_com:
    data_train_input_rt.append(rt_data[i].to_numpy()[:,1:3+rt])
  data_train_output_rt = []
  for i in out_list:
    data_train_output_rt.append(rt_data[i].to_numpy()[:,1:3+rt])

  # dirmo3
  X_train_list = []
  y_train_list = []
  X_test_list = []
  y_test_list = []

  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirmo3(data_train_input[i],rt)[0])
    X_test_stack.append(make_dirmo3(data_test_input[i],rt)[0])
    if rt >=3:
      X_train_stack[i] = np.concatenate([X_train_stack[i],make_dirmo3(data_train_input_rt[i][0:1,0:rt+2],rt)[0]])
  y_train_stack.append(make_dirmo3(data_train_output[out_list.index(tn)],rt)[1])
  if rt >=3:
    y_train_stack[0] = np.concatenate([y_train_stack[0],make_dirmo3(data_train_output_rt[out_list.index(tn)][0:1,0:rt+2],rt)[1]])
  y_test_stack.append(make_dirmo3(data_test_output[out_list.index(tn)],rt)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  # dirmo2
  X_train_stack = []
  y_train_stack = []
  X_test_stack = []
  y_test_stack = []
  for i in range(len(all_vcd_com)):
    X_train_stack.append(make_dirmo2(data_train_input[i],rt)[0])
    X_test_stack.append(make_dirmo2(data_test_input[i],rt)[0])
    if rt >=4:
      X_train_stack[i] = np.concatenate([X_train_stack[i],make_dirmo2(data_train_input_rt[i][0:1,0:rt+2],rt)[0]])
  y_train_stack.append(make_dirmo2(data_train_output[out_list.index(tn)],rt)[1])
  if rt >=4:
    y_train_stack[0] = np.concatenate([y_train_stack[0],make_dirmo2(data_train_output_rt[out_list.index(tn)][0:1,0:rt+2],rt)[1]])
  y_test_stack.append(make_dirmo2(data_test_output[out_list.index(tn)],rt)[1])
  X_train_list.append(np.hstack(X_train_stack))
  y_train_list.append(np.hstack(y_train_stack))
  X_test_list.append(np.hstack(X_test_stack))
  y_test_list.append(np.hstack(y_test_stack))

  X_train_s_list = []
  X_test_s_list = []
  scaler_list = []
  for i in range(len(X_train_list)):
    scaler = StandardScaler()
    X_train_s_list.append(scaler.fit_transform(X_train_list[i]))
    X_test_s_list.append(scaler.transform(X_test_list[i]))
    scaler_list.append(scaler)

  if mn in [13,14,15]:
    for i in range(len(X_train_s_list)):
      X_train_s_list[i] = X_train_s_list[i].reshape(X_train_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)
      X_test_s_list[i]  = X_test_s_list[i].reshape(X_test_s_list[i].shape[0],len(all_vcd_com),-1).transpose(0,2,1)

  models = []
  for i in range(len(X_train_s_list)):
    seed_fix()
    models.append(algorithm_selection(mn,X_train_s_list[i].shape[1],y_train_list[i].shape[1],len(all_vcd_com),len(all_vcd_com),y_train_list[i].shape[1],X_test_s_list[i].shape[1],Learning_rate))

  if mn in range(12):
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i])
  elif mn in [13,14,15,16]:
    for i in range(len(X_train_s_list)):
      seed_fix()
      models[i].fit(X_train_s_list[i],y_train_list[i],epochs=100,verbose=0)

  conditions = []
  ten_rd = np.concatenate(data_test_input)[:,0:2+rt].reshape(-1,4,2+rt)
  conditions.append(ten_rd[:,0,:])
  conditions.append(ten_rd[:,1,:])
  conditions.append(ten_rd[:,2,:])
  conditions.append(ten_rd[:,3,:])
  VCD_tem = conditions.copy()

  conditions = []
  conditions.append(raw_data[tn][condition_test].to_numpy()[0,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[1,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[2,1:3+rt])
  conditions.append(raw_data[tn][condition_test].to_numpy()[3,1:3+rt])
  y_pred_all = conditions.copy()

  for cn in range(4):
    y_pred_all[cn] = np.concatenate([y_pred_all[cn],np.zeros(shape=(8-y_pred_all[cn].shape[0]))],0)

  dir3_f = [3,4,5,6]
  dir3_b = [6,6,7,7]
  dir2_f = [6,6,7,7]
  dir2_b = [8,8,8,8]

  if mn in range(13):
    # dirmo
    for cn in range(4):
      y_pred_all[cn][dir3_f[rt-1]:dir3_b[rt-1]] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,0:3].reshape(1,-1))).reshape(dir3_b[rt-1]-dir3_f[rt-1])
      y_pred_all[cn][dir2_f[rt-1]:dir2_b[rt-1]] = models[1].predict(scaler_list[1].transform(VCD_tem[cn][:,0:3].reshape(1,-1))).reshape(dir2_b[rt-1]-dir2_f[rt-1])
  elif mn in [13,14,15]:
    # dirmo RNN
    for cn in range(4):
      y_pred_all[cn][dir3_f[rt-1]:dir3_b[rt-1]] = models[0].predict(scaler_list[0].transform(VCD_tem[cn][:,0:3].reshape(1,-1)).reshape(1,VCD_tem[cn][:,0:3].shape[0],VCD_tem[cn][:,0:3].shape[1]).transpose(0,2,1),verbose=0).reshape(dir3_b[rt-1]-dir3_f[rt-1])
      y_pred_all[cn][dir2_f[rt-1]:dir2_b[rt-1]] = models[1].predict(scaler_list[1].transform(VCD_tem[cn][:,0:3].reshape(1,-1)).reshape(1,VCD_tem[cn][:,0:3].shape[0],VCD_tem[cn][:,0:3].shape[1]).transpose(0,2,1),verbose=0).reshape(dir2_b[rt-1]-dir2_f[rt-1])

  
  return y_pred_all, np.concatenate([y_pred_all[0][2+rt:8],y_pred_all[1][2+rt:8],y_pred_all[2][2+rt:8],y_pred_all[3][2+rt:8]]), models

##########################################################################################################
##########################################################################################################
##########################################################################################################


def forecasting_selection(rt,ra_data,f,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate):
  if f == 0:
    return recursive_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate)
  elif f == 1:
    return direct_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate)
  elif f == 2:
    return mimo_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate)
  elif f == 3:
    return dirrec_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate)
  elif f == 4:
    return dirmo_predict(rt,rt_data,all_vcd_com,mn,raw_data,condition_train,condition_test,Learning_rate)


# **Evaluation Process**

In [6]:
start = time.time()  # 시작 시간 저장
warnings.filterwarnings("ignore")
### data 준비 ###
# one step input
seed_fix()
data_input = []
data_input.append(pd.read_csv("input_vcd.csv"))
data_input.append(pd.read_csv("input_viability.csv"))
data_input.append(pd.read_csv("input_glucose.csv"))
data_input.append(pd.read_csv("input_lactate.csv"))
data_input.append(pd.read_csv("input_ammonia.csv"))
data_input.append(pd.read_csv("input_do.csv"))
data_input.append(pd.read_csv("input_ph.csv"))
data_input.append(pd.read_csv("input_igg.csv"))

# one step output
data_output = []
data_output.append(pd.read_csv("output_vcd.csv"))
data_output.append(pd.read_csv("input_viability.csv"))
data_output.append(pd.read_csv("output_glucose.csv"))
data_output.append(pd.read_csv("output_lactate.csv"))
data_output.append(pd.read_csv("output_ammonia.csv"))
data_output.append(pd.read_csv("input_do.csv"))
data_output.append(pd.read_csv("input_ph.csv"))
data_output.append(pd.read_csv("output_igg.csv"))

# multistep data
raw_data = []
raw_data.append(pd.read_csv('vcd.csv'))
raw_data.append(pd.read_csv('viability.csv'))
raw_data.append(pd.read_csv('glucose.csv'))
raw_data.append(pd.read_csv('lactate.csv'))
raw_data.append(pd.read_csv('ammonia.csv'))
raw_data.append(pd.read_csv('do.csv'))
raw_data.append(pd.read_csv('ph.csv'))
raw_data.append(pd.read_csv('igg.csv'))

# RT data
rt_data = []
rt_data.append(pd.read_csv('vcd_rt.csv'))
rt_data.append(pd.read_csv('viability_rt.csv'))
rt_data.append(pd.read_csv('glucose_rt.csv'))
rt_data.append(pd.read_csv('lactate_rt.csv'))
rt_data.append(pd.read_csv('ammonia_rt.csv'))
rt_data.append(pd.read_csv('do_rt.csv'))
rt_data.append(pd.read_csv('ph_rt.csv'))
rt_data.append(pd.read_csv('igg_rt.csv'))


# test batches for each condition
condition_test1 = "A1_2"
condition_test2 = "B1_3"
condition_test3 = "A2_6"
condition_test4 = "B2_7"
condition_test_os  = (data_input[0].condition == condition_test1) + (data_input[0].condition == condition_test2) + (data_input[0].condition == condition_test3) + (data_input[0].condition == condition_test4)
condition_train_os = ~condition_test_os
condition_train_ms = ~((raw_data[0].condition == condition_test1) + (raw_data[0].condition == condition_test2) + (raw_data[0].condition == condition_test3) + (raw_data[0].condition == condition_test4))
condition_test_ms  = (raw_data[0].condition == condition_test1) + (raw_data[0].condition == condition_test2) + (raw_data[0].condition == condition_test3) + (raw_data[0].condition == condition_test4)

# permutation item and etc..
item = [0,1,2,3,4,5,6,7]
model_name=np.array(["LR","LASSO","EN","KNN","CART","GBM","XGB","Ada","SVM","RF","PLS","GPR","ANN","RNN","LSTM","GRU"])
forecasting_strategy = np.array(["Recursive",'Direct','MIMO','DirRec','DIRMO'])

####################################################################################################################################
### forecasting strategy evaluation ################################################################################################
####################################################################################################################################

all_vcd_com = [0,1,2,3,4,5,6,7]

X_train_os_list = []
X_test_os_list = []
for j in all_vcd_com:
  X_train_os_list.append(data_input[j][condition_train_os].to_numpy()[:,1:4])
  X_test_os_list.append(data_input[j][condition_test_os].to_numpy()[:,1:4])
X_train_os = np.concatenate(X_train_os_list,1)
X_test_os = np.concatenate(X_test_os_list,1)

scaler_os = StandardScaler()
X_train_s_os = scaler_os.fit_transform(X_train_os)
X_test_s_os = scaler_os.transform(X_test_os)
X_train_s_rs_os  = X_train_s_os.reshape(X_train_s_os.shape[0],len(all_vcd_com),-1).transpose(0,2,1)
X_test_s_rs_os   = X_test_s_os.reshape(X_test_s_os.shape[0],len(all_vcd_com),-1).transpose(0,2,1)

y_train_os = data_output[tn][condition_train_os].to_numpy()[:,1:2]
y_test_os  = data_output[tn][condition_test_os].to_numpy()[:,1:2]

rty_list = list(range(20))
for i in range(4):
  for j in range(rt-1):
    rty_list.remove(5*i+j)

error_table_f_mae = []
error_table_f_mape = []
error_table_f_mse = []
error_table_f_rmse = []
error_table_f_rse = []
y_pred_os_trajectory_f = []
y_pred_ms_trajectory_f = []
for mn in range(16):
  error_table_f_mae.append([])
  error_table_f_mape.append([])
  error_table_f_mse.append([])
  error_table_f_rmse.append([])
  error_table_f_rse.append([])
  y_pred_os_trajectory_f.append([])
  y_pred_ms_trajectory_f.append([])


y_test_trajectory = y_test_os[rty_list]

for mn in range(16):
  for f in range(5):
    y_pred_ms_trajectory_f[mn].append(forecasting_selection(rt,rt_data,f,all_vcd_com,mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1])
    error_table_f_mae[mn].append(mae(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_mape[mn].append(mape(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_mse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_rmse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f],squared=False))
    error_table_f_rse[mn].append(rse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))

best_forecasting_no = []
for mn in range(16):
  best_forecasting_no.append(error_table_f_mape[mn].index(min(error_table_f_mape[mn])))


############################################################################################################################################################
### model input evaluation #################################################################################################################################
############################################################################################################################################################

error_table_i_mae = []
error_table_i_mape = []
error_table_i_mse = []
error_table_i_rmse = []
error_table_i_rse = []
best_error_vcd_mae = []
best_error_vcd_mape = []
best_error_vcd_mse = []
best_error_vcd_rmse = []
best_error_vcd_rse = []
best_error_vcd_com = []
best_error_vcd_pred = []

for mn in range(16):
  error_table_i_mae.append([])
  error_table_i_mape.append([])
  error_table_i_mse.append([])
  error_table_i_rmse.append([])
  error_table_i_rse.append([])
  best_error_vcd_mae.append(100)
  best_error_vcd_mape.append(100)
  best_error_vcd_mse.append(100)
  best_error_vcd_rmse.append(100)
  best_error_vcd_rse.append(100)
  best_error_vcd_com.append([])
  best_error_vcd_pred.append([])



com_list = []
for k in range(8):
  com = list(combinations(item,k+1))
  for i in range(len(com)):
    com_list.append(com[i])


for i in range(len(com_list)):
  for mn in range(16):
    y_pred_ms_trajectory_i = forecasting_selection(rt,rt_data,best_forecasting_no[mn],com_list[i],mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1]
    error_table_i_mae[mn].append(mae(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_mape[mn].append(mape(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_mse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_rmse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_i,squared=False))
    error_table_i_rse[mn].append(rse(y_test_trajectory,y_pred_ms_trajectory_i))

    if mape(y_test_trajectory,y_pred_ms_trajectory_i) < best_error_vcd_mape[mn]:
      best_error_vcd_mae[mn] = mae(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_mape[mn] = mape(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_mse[mn] = mse(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_rmse[mn] = mse(y_test_trajectory,y_pred_ms_trajectory_i,squared=False)
      best_error_vcd_rse[mn] = rse(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_com[mn] = com_list[i]
      best_error_vcd_pred[mn] = y_pred_ms_trajectory_i
  

all_error_vcd_mae = []
all_error_vcd_mape = []
all_error_vcd_mse = []
all_error_vcd_rmse = []
all_error_vcd_rse = []
all_error_vcd_com = []
all_error_vcd_pred = []
for mn in range(16):
  all_error_vcd_mae.append(100)
  all_error_vcd_mape.append(100)
  all_error_vcd_mse.append(100)
  all_error_vcd_rmse.append(100)
  all_error_vcd_rse.append(100)
  all_error_vcd_com.append(100)
  all_error_vcd_pred.append(100)
for mn in range(16):
  all_error_vcd_pred[mn] = forecasting_selection(rt,rt_data,best_forecasting_no[mn],com_list[i],mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1]  
  all_error_vcd_mae[mn] = mae(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_mape[mn] = mape(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_mse[mn] = mse(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_rmse[mn] = mse(y_test_trajectory,all_error_vcd_pred[mn],squared=False)
  all_error_vcd_rse[mn] = rse(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_com[mn] = com_list[i]


############################################################################################################################################
### Algorithm evaluation ###################################################################################################################
############################################################################################################################################

error_table_a_mae   = best_error_vcd_mae.copy()
error_table_a_mape  = best_error_vcd_mape.copy()
error_table_a_mse   = best_error_vcd_mse.copy()
error_table_a_rmse  = best_error_vcd_rmse.copy()
error_table_a_rse   = best_error_vcd_rse.copy()
y_pred_trajectory_A = best_error_vcd_pred.copy()

best_algorithm_no = error_table_a_mape.index(min(error_table_a_mape))

for mn in range(12):
  print('F: the minimum MAPE of',model_name[mn],'is',min(error_table_f_mape[mn])*100,'% and the forecasting strategy is ',forecasting_strategy[error_table_f_mape[mn].index(min(error_table_f_mape[mn]))])
for mn in range(12):
  print('I: the MAPE of best input combination is ',min(error_table_i_mape[mn])*100,'% and the best input combination is ',best_error_vcd_com[mn])
print('A: the minimum MAPE is ',min(error_table_a_mape)*100,'% and the algorithms is ',model_name[best_algorithm_no],'with',forecasting_strategy[best_forecasting_no[best_algorithm_no]])

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

[11:54:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[11:54:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

1/1 [==============================] - 0s 26ms/step


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 46ms/step
[12:08:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:08:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:08:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1/1 [==============================] - 0s 28ms/step
[12:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[12:09:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
1/1 [==============================] - 0s 43ms/step
[12:10:08] WARNING: /workspace/src/objective/r

In [7]:
print(best_forecasting_no[best_algorithm_no])
print(best_error_vcd_com[best_algorithm_no])
print(best_algorithm_no)
print(y_test_trajectory.T)
print(forecasting_selection(rt,rt_data,best_forecasting_no[best_algorithm_no],best_error_vcd_com[best_algorithm_no],best_algorithm_no,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1])

2
(2, 4)
4
[[0.353 0.619 0.7 0.774 0.514 0.559 0.599 0.615 0.444 0.71 0.904 1.047
  0.351 0.576 0.735 0.874]]
[0.36 0.64 0.73 0.786 0.498 0.536 0.593 0.608 0.471 0.747 0.893 1.034
 0.377 0.547 0.749 0.888]


In [8]:
rt=3

start = time.time()  # 시작 시간 저장
warnings.filterwarnings("ignore")
### data 준비 ###
# one step input
seed_fix()
data_input = []
data_input.append(pd.read_csv("input_vcd.csv"))
data_input.append(pd.read_csv("input_viability.csv"))
data_input.append(pd.read_csv("input_glucose.csv"))
data_input.append(pd.read_csv("input_lactate.csv"))
data_input.append(pd.read_csv("input_ammonia.csv"))
data_input.append(pd.read_csv("input_do.csv"))
data_input.append(pd.read_csv("input_ph.csv"))
data_input.append(pd.read_csv("input_igg.csv"))

# one step output
data_output = []
data_output.append(pd.read_csv("output_vcd.csv"))
data_output.append(pd.read_csv("input_viability.csv"))
data_output.append(pd.read_csv("output_glucose.csv"))
data_output.append(pd.read_csv("output_lactate.csv"))
data_output.append(pd.read_csv("output_ammonia.csv"))
data_output.append(pd.read_csv("input_do.csv"))
data_output.append(pd.read_csv("input_ph.csv"))
data_output.append(pd.read_csv("output_igg.csv"))

# multistep data
raw_data = []
raw_data.append(pd.read_csv('vcd.csv'))
raw_data.append(pd.read_csv('viability.csv'))
raw_data.append(pd.read_csv('glucose.csv'))
raw_data.append(pd.read_csv('lactate.csv'))
raw_data.append(pd.read_csv('ammonia.csv'))
raw_data.append(pd.read_csv('do.csv'))
raw_data.append(pd.read_csv('ph.csv'))
raw_data.append(pd.read_csv('igg.csv'))

# RT data
rt_data = []
rt_data.append(pd.read_csv('vcd_rt.csv'))
rt_data.append(pd.read_csv('viability_rt.csv'))
rt_data.append(pd.read_csv('glucose_rt.csv'))
rt_data.append(pd.read_csv('lactate_rt.csv'))
rt_data.append(pd.read_csv('ammonia_rt.csv'))
rt_data.append(pd.read_csv('do_rt.csv'))
rt_data.append(pd.read_csv('ph_rt.csv'))
rt_data.append(pd.read_csv('igg_rt.csv'))


# test batches for each condition
condition_test1 = "A1_2"
condition_test2 = "B1_3"
condition_test3 = "A2_6"
condition_test4 = "B2_7"
condition_test_os  = (data_input[0].condition == condition_test1) + (data_input[0].condition == condition_test2) + (data_input[0].condition == condition_test3) + (data_input[0].condition == condition_test4)
condition_train_os = ~condition_test_os
condition_train_ms = ~((raw_data[0].condition == condition_test1) + (raw_data[0].condition == condition_test2) + (raw_data[0].condition == condition_test3) + (raw_data[0].condition == condition_test4))
condition_test_ms  = (raw_data[0].condition == condition_test1) + (raw_data[0].condition == condition_test2) + (raw_data[0].condition == condition_test3) + (raw_data[0].condition == condition_test4)

# permutation item and etc..
item = [0,1,2,3,4,5,6,7]
model_name=np.array(["LR","LASSO","EN","KNN","CART","GBM","XGB","Ada","SVM","RF","PLS","GPR","ANN","RNN","LSTM","GRU"])
forecasting_strategy = np.array(["Recursive",'Direct','MIMO','DirRec','DIRMO'])

####################################################################################################################################
### forecasting strategy evaluation ################################################################################################
####################################################################################################################################

all_vcd_com = [0,1,2,3,4,5,6,7]

X_train_os_list = []
X_test_os_list = []
for j in all_vcd_com:
  X_train_os_list.append(data_input[j][condition_train_os].to_numpy()[:,1:4])
  X_test_os_list.append(data_input[j][condition_test_os].to_numpy()[:,1:4])
X_train_os = np.concatenate(X_train_os_list,1)
X_test_os = np.concatenate(X_test_os_list,1)

scaler_os = StandardScaler()
X_train_s_os = scaler_os.fit_transform(X_train_os)
X_test_s_os = scaler_os.transform(X_test_os)
X_train_s_rs_os  = X_train_s_os.reshape(X_train_s_os.shape[0],len(all_vcd_com),-1).transpose(0,2,1)
X_test_s_rs_os   = X_test_s_os.reshape(X_test_s_os.shape[0],len(all_vcd_com),-1).transpose(0,2,1)

y_train_os = data_output[tn][condition_train_os].to_numpy()[:,1:2]
y_test_os  = data_output[tn][condition_test_os].to_numpy()[:,1:2]

rty_list = list(range(20))
for i in range(4):
  for j in range(rt-1):
    rty_list.remove(5*i+j)

error_table_f_mae = []
error_table_f_mape = []
error_table_f_mse = []
error_table_f_rmse = []
error_table_f_rse = []
y_pred_os_trajectory_f = []
y_pred_ms_trajectory_f = []
for mn in range(16):
  error_table_f_mae.append([])
  error_table_f_mape.append([])
  error_table_f_mse.append([])
  error_table_f_rmse.append([])
  error_table_f_rse.append([])
  y_pred_os_trajectory_f.append([])
  y_pred_ms_trajectory_f.append([])


y_test_trajectory = y_test_os[rty_list]

for mn in range(16):
  for f in range(5):
    y_pred_ms_trajectory_f[mn].append(forecasting_selection(rt,rt_data,f,all_vcd_com,mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1])
    error_table_f_mae[mn].append(mae(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_mape[mn].append(mape(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_mse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_rmse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f],squared=False))
    error_table_f_rse[mn].append(rse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))

best_forecasting_no = []
for mn in range(16):
  best_forecasting_no.append(error_table_f_mape[mn].index(min(error_table_f_mape[mn])))


############################################################################################################################################################
### model input evaluation #################################################################################################################################
############################################################################################################################################################

error_table_i_mae = []
error_table_i_mape = []
error_table_i_mse = []
error_table_i_rmse = []
error_table_i_rse = []
best_error_vcd_mae = []
best_error_vcd_mape = []
best_error_vcd_mse = []
best_error_vcd_rmse = []
best_error_vcd_rse = []
best_error_vcd_com = []
best_error_vcd_pred = []

for mn in range(16):
  error_table_i_mae.append([])
  error_table_i_mape.append([])
  error_table_i_mse.append([])
  error_table_i_rmse.append([])
  error_table_i_rse.append([])
  best_error_vcd_mae.append(100)
  best_error_vcd_mape.append(100)
  best_error_vcd_mse.append(100)
  best_error_vcd_rmse.append(100)
  best_error_vcd_rse.append(100)
  best_error_vcd_com.append([])
  best_error_vcd_pred.append([])



com_list = []
for k in range(8):
  com = list(combinations(item,k+1))
  for i in range(len(com)):
    com_list.append(com[i])


for i in range(len(com_list)):
  for mn in range(16):
    y_pred_ms_trajectory_i = forecasting_selection(rt,rt_data,best_forecasting_no[mn],com_list[i],mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1]
    error_table_i_mae[mn].append(mae(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_mape[mn].append(mape(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_mse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_rmse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_i,squared=False))
    error_table_i_rse[mn].append(rse(y_test_trajectory,y_pred_ms_trajectory_i))

    if mape(y_test_trajectory,y_pred_ms_trajectory_i) < best_error_vcd_mape[mn]:
      best_error_vcd_mae[mn] = mae(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_mape[mn] = mape(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_mse[mn] = mse(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_rmse[mn] = mse(y_test_trajectory,y_pred_ms_trajectory_i,squared=False)
      best_error_vcd_rse[mn] = rse(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_com[mn] = com_list[i]
      best_error_vcd_pred[mn] = y_pred_ms_trajectory_i
  

all_error_vcd_mae = []
all_error_vcd_mape = []
all_error_vcd_mse = []
all_error_vcd_rmse = []
all_error_vcd_rse = []
all_error_vcd_com = []
all_error_vcd_pred = []
for mn in range(16):
  all_error_vcd_mae.append(100)
  all_error_vcd_mape.append(100)
  all_error_vcd_mse.append(100)
  all_error_vcd_rmse.append(100)
  all_error_vcd_rse.append(100)
  all_error_vcd_com.append(100)
  all_error_vcd_pred.append(100)
for mn in range(16):
  all_error_vcd_pred[mn] = forecasting_selection(rt,rt_data,best_forecasting_no[mn],com_list[i],mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1]  
  all_error_vcd_mae[mn] = mae(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_mape[mn] = mape(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_mse[mn] = mse(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_rmse[mn] = mse(y_test_trajectory,all_error_vcd_pred[mn],squared=False)
  all_error_vcd_rse[mn] = rse(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_com[mn] = com_list[i]


############################################################################################################################################
### Algorithm evaluation ###################################################################################################################
############################################################################################################################################

error_table_a_mae   = best_error_vcd_mae.copy()
error_table_a_mape  = best_error_vcd_mape.copy()
error_table_a_mse   = best_error_vcd_mse.copy()
error_table_a_rmse  = best_error_vcd_rmse.copy()
error_table_a_rse   = best_error_vcd_rse.copy()
y_pred_trajectory_A = best_error_vcd_pred.copy()

best_algorithm_no = error_table_a_mape.index(min(error_table_a_mape))

for mn in range(12):
  print('F: the minimum MAPE of',model_name[mn],'is',min(error_table_f_mape[mn])*100,'% and the forecasting strategy is ',forecasting_strategy[error_table_f_mape[mn].index(min(error_table_f_mape[mn]))])
for mn in range(12):
  print('I: the MAPE of best input combination is ',min(error_table_i_mape[mn])*100,'% and the best input combination is ',best_error_vcd_com[mn])
print('A: the minimum MAPE is ',min(error_table_a_mape)*100,'% and the algorithms is ',model_name[best_algorithm_no],'with',forecasting_strategy[best_forecasting_no[best_algorithm_no]])

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

[15:04:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:04:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [9]:
print(best_forecasting_no[best_algorithm_no])
print(best_error_vcd_com[best_algorithm_no])
print(best_algorithm_no)
print(y_test_trajectory.T)
print(forecasting_selection(rt,rt_data,best_forecasting_no[best_algorithm_no],best_error_vcd_com[best_algorithm_no],best_algorithm_no,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1])

0
(0, 1, 3, 5, 6)
4
[[0.619 0.7 0.774 0.559 0.599 0.615 0.71 0.904 1.047 0.576 0.735 0.874]]
[array([0.64]) array([0.73]) array([0.786]) array([0.536]) array([0.593])
 array([0.608]) array([0.747]) array([0.893]) array([1.034])
 array([0.592]) array([0.738]) array([0.888])]


In [10]:
rt=4

start = time.time()  # 시작 시간 저장
warnings.filterwarnings("ignore")
### data 준비 ###
# one step input
seed_fix()
data_input = []
data_input.append(pd.read_csv("input_vcd.csv"))
data_input.append(pd.read_csv("input_viability.csv"))
data_input.append(pd.read_csv("input_glucose.csv"))
data_input.append(pd.read_csv("input_lactate.csv"))
data_input.append(pd.read_csv("input_ammonia.csv"))
data_input.append(pd.read_csv("input_do.csv"))
data_input.append(pd.read_csv("input_ph.csv"))
data_input.append(pd.read_csv("input_igg.csv"))

# one step output
data_output = []
data_output.append(pd.read_csv("output_vcd.csv"))
data_output.append(pd.read_csv("input_viability.csv"))
data_output.append(pd.read_csv("output_glucose.csv"))
data_output.append(pd.read_csv("output_lactate.csv"))
data_output.append(pd.read_csv("output_ammonia.csv"))
data_output.append(pd.read_csv("input_do.csv"))
data_output.append(pd.read_csv("input_ph.csv"))
data_output.append(pd.read_csv("output_igg.csv"))

# multistep data
raw_data = []
raw_data.append(pd.read_csv('vcd.csv'))
raw_data.append(pd.read_csv('viability.csv'))
raw_data.append(pd.read_csv('glucose.csv'))
raw_data.append(pd.read_csv('lactate.csv'))
raw_data.append(pd.read_csv('ammonia.csv'))
raw_data.append(pd.read_csv('do.csv'))
raw_data.append(pd.read_csv('ph.csv'))
raw_data.append(pd.read_csv('igg.csv'))

# RT data
rt_data = []
rt_data.append(pd.read_csv('vcd_rt.csv'))
rt_data.append(pd.read_csv('viability_rt.csv'))
rt_data.append(pd.read_csv('glucose_rt.csv'))
rt_data.append(pd.read_csv('lactate_rt.csv'))
rt_data.append(pd.read_csv('ammonia_rt.csv'))
rt_data.append(pd.read_csv('do_rt.csv'))
rt_data.append(pd.read_csv('ph_rt.csv'))
rt_data.append(pd.read_csv('igg_rt.csv'))


# test batches for each condition
condition_test1 = "A1_2"
condition_test2 = "B1_3"
condition_test3 = "A2_6"
condition_test4 = "B2_7"
condition_test_os  = (data_input[0].condition == condition_test1) + (data_input[0].condition == condition_test2) + (data_input[0].condition == condition_test3) + (data_input[0].condition == condition_test4)
condition_train_os = ~condition_test_os
condition_train_ms = ~((raw_data[0].condition == condition_test1) + (raw_data[0].condition == condition_test2) + (raw_data[0].condition == condition_test3) + (raw_data[0].condition == condition_test4))
condition_test_ms  = (raw_data[0].condition == condition_test1) + (raw_data[0].condition == condition_test2) + (raw_data[0].condition == condition_test3) + (raw_data[0].condition == condition_test4)

# permutation item and etc..
item = [0,1,2,3,4,5,6,7]
model_name=np.array(["LR","LASSO","EN","KNN","CART","GBM","XGB","Ada","SVM","RF","PLS","GPR","ANN","RNN","LSTM","GRU"])
forecasting_strategy = np.array(["Recursive",'Direct','MIMO','DirRec','DIRMO'])

####################################################################################################################################
### forecasting strategy evaluation ################################################################################################
####################################################################################################################################

all_vcd_com = [0,1,2,3,4,5,6,7]

X_train_os_list = []
X_test_os_list = []
for j in all_vcd_com:
  X_train_os_list.append(data_input[j][condition_train_os].to_numpy()[:,1:4])
  X_test_os_list.append(data_input[j][condition_test_os].to_numpy()[:,1:4])
X_train_os = np.concatenate(X_train_os_list,1)
X_test_os = np.concatenate(X_test_os_list,1)

scaler_os = StandardScaler()
X_train_s_os = scaler_os.fit_transform(X_train_os)
X_test_s_os = scaler_os.transform(X_test_os)
X_train_s_rs_os  = X_train_s_os.reshape(X_train_s_os.shape[0],len(all_vcd_com),-1).transpose(0,2,1)
X_test_s_rs_os   = X_test_s_os.reshape(X_test_s_os.shape[0],len(all_vcd_com),-1).transpose(0,2,1)

y_train_os = data_output[tn][condition_train_os].to_numpy()[:,1:2]
y_test_os  = data_output[tn][condition_test_os].to_numpy()[:,1:2]

rty_list = list(range(20))
for i in range(4):
  for j in range(rt-1):
    rty_list.remove(5*i+j)

error_table_f_mae = []
error_table_f_mape = []
error_table_f_mse = []
error_table_f_rmse = []
error_table_f_rse = []
y_pred_os_trajectory_f = []
y_pred_ms_trajectory_f = []
for mn in range(16):
  error_table_f_mae.append([])
  error_table_f_mape.append([])
  error_table_f_mse.append([])
  error_table_f_rmse.append([])
  error_table_f_rse.append([])
  y_pred_os_trajectory_f.append([])
  y_pred_ms_trajectory_f.append([])


y_test_trajectory = y_test_os[rty_list]

for mn in range(16):
  for f in range(5):
    y_pred_ms_trajectory_f[mn].append(forecasting_selection(rt,rt_data,f,all_vcd_com,mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1])
    error_table_f_mae[mn].append(mae(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_mape[mn].append(mape(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_mse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))
    error_table_f_rmse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f],squared=False))
    error_table_f_rse[mn].append(rse(y_test_trajectory,y_pred_ms_trajectory_f[mn][f]))

best_forecasting_no = []
for mn in range(16):
  best_forecasting_no.append(error_table_f_mape[mn].index(min(error_table_f_mape[mn])))


############################################################################################################################################################
### model input evaluation #################################################################################################################################
############################################################################################################################################################

error_table_i_mae = []
error_table_i_mape = []
error_table_i_mse = []
error_table_i_rmse = []
error_table_i_rse = []
best_error_vcd_mae = []
best_error_vcd_mape = []
best_error_vcd_mse = []
best_error_vcd_rmse = []
best_error_vcd_rse = []
best_error_vcd_com = []
best_error_vcd_pred = []

for mn in range(16):
  error_table_i_mae.append([])
  error_table_i_mape.append([])
  error_table_i_mse.append([])
  error_table_i_rmse.append([])
  error_table_i_rse.append([])
  best_error_vcd_mae.append(100)
  best_error_vcd_mape.append(100)
  best_error_vcd_mse.append(100)
  best_error_vcd_rmse.append(100)
  best_error_vcd_rse.append(100)
  best_error_vcd_com.append([])
  best_error_vcd_pred.append([])



com_list = []
for k in range(8):
  com = list(combinations(item,k+1))
  for i in range(len(com)):
    com_list.append(com[i])


for i in range(len(com_list)):
  for mn in range(16):
    y_pred_ms_trajectory_i = forecasting_selection(rt,rt_data,best_forecasting_no[mn],com_list[i],mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1]
    error_table_i_mae[mn].append(mae(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_mape[mn].append(mape(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_mse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_i))
    error_table_i_rmse[mn].append(mse(y_test_trajectory,y_pred_ms_trajectory_i,squared=False))
    error_table_i_rse[mn].append(rse(y_test_trajectory,y_pred_ms_trajectory_i))

    if mape(y_test_trajectory,y_pred_ms_trajectory_i) < best_error_vcd_mape[mn]:
      best_error_vcd_mae[mn] = mae(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_mape[mn] = mape(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_mse[mn] = mse(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_rmse[mn] = mse(y_test_trajectory,y_pred_ms_trajectory_i,squared=False)
      best_error_vcd_rse[mn] = rse(y_test_trajectory,y_pred_ms_trajectory_i)
      best_error_vcd_com[mn] = com_list[i]
      best_error_vcd_pred[mn] = y_pred_ms_trajectory_i
  

all_error_vcd_mae = []
all_error_vcd_mape = []
all_error_vcd_mse = []
all_error_vcd_rmse = []
all_error_vcd_rse = []
all_error_vcd_com = []
all_error_vcd_pred = []
for mn in range(16):
  all_error_vcd_mae.append(100)
  all_error_vcd_mape.append(100)
  all_error_vcd_mse.append(100)
  all_error_vcd_rmse.append(100)
  all_error_vcd_rse.append(100)
  all_error_vcd_com.append(100)
  all_error_vcd_pred.append(100)
for mn in range(16):
  all_error_vcd_pred[mn] = forecasting_selection(rt,rt_data,best_forecasting_no[mn],com_list[i],mn,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1]  
  all_error_vcd_mae[mn] = mae(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_mape[mn] = mape(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_mse[mn] = mse(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_rmse[mn] = mse(y_test_trajectory,all_error_vcd_pred[mn],squared=False)
  all_error_vcd_rse[mn] = rse(y_test_trajectory,all_error_vcd_pred[mn])
  all_error_vcd_com[mn] = com_list[i]


############################################################################################################################################
### Algorithm evaluation ###################################################################################################################
############################################################################################################################################

error_table_a_mae   = best_error_vcd_mae.copy()
error_table_a_mape  = best_error_vcd_mape.copy()
error_table_a_mse   = best_error_vcd_mse.copy()
error_table_a_rmse  = best_error_vcd_rmse.copy()
error_table_a_rse   = best_error_vcd_rse.copy()
y_pred_trajectory_A = best_error_vcd_pred.copy()

best_algorithm_no = error_table_a_mape.index(min(error_table_a_mape))

for mn in range(12):
  print('F: the minimum MAPE of',model_name[mn],'is',min(error_table_f_mape[mn])*100,'% and the forecasting strategy is ',forecasting_strategy[error_table_f_mape[mn].index(min(error_table_f_mape[mn]))])
for mn in range(12):
  print('I: the MAPE of best input combination is ',min(error_table_i_mape[mn])*100,'% and the best input combination is ',best_error_vcd_com[mn])
print('A: the minimum MAPE is ',min(error_table_a_mape)*100,'% and the algorithms is ',model_name[best_algorithm_no],'with',forecasting_strategy[best_forecasting_no[best_algorithm_no]])

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

[16:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:07:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [11]:
print(best_forecasting_no[best_algorithm_no])
print(best_error_vcd_com[best_algorithm_no])
print(best_algorithm_no)
print(y_test_trajectory.T)
print(forecasting_selection(rt,rt_data,best_forecasting_no[best_algorithm_no],best_error_vcd_com[best_algorithm_no],best_algorithm_no,raw_data,condition_train_ms,condition_test_ms,hp_lr[mn])[1])

0
(0, 3, 4, 6, 7)
4
[[0.7 0.774 0.599 0.615 0.904 1.047 0.735 0.874]]
[array([0.73]) array([0.786]) array([0.593]) array([0.608]) array([0.893])
 array([1.034]) array([0.738]) array([0.882])]
